In [1]:
import urllib
import requests
import bs4
import selenium

# 1. 댓글 수집하기

In [2]:
url = 'http://corners.auction.co.kr/AllKill/AllDay.aspx'

## 1. urllib을 이용한 html page 가져오기

In [3]:
file, http = urllib.request.urlretrieve(url, './data/auction_list.html')